In [2]:
import json

In [18]:
def get_data_from_jsons():
    leinings = []
    for i in range(1,5):
        filename = 'reference texts/leyning'+str(i)+'.json'
        with open(filename,encoding='utf-8') as json_file:
            table = json.load(json_file)
            for item in table["items"]:
                leining = {"name":item["name"]["en"]}
                if "weekday" in item.keys():
                    leining["type"]="weekday"
                    leining["reading"] = item["weekday"]
                elif "fullkriyah" in item.keys():
                    if len(item['fullkriyah']) > 5:
                        leining["type"] = "shabYT"
                    elif len(item['fullkriyah']) <= 5:
                        leining["type"] = "minorHol"
                    leining["reading"] = item["fullkriyah"]
                    if "haft" in item.keys():
                        leining["reading"]["haft"] = item["haft"]
                        if "reason" in item.keys():
                            leining["reading"]["haftname"] = item["reason"]
                        else:
                            leining['reading']['haftname'] = leining['name']
                else:
                    continue
                if leining not in leinings:
                    leinings.append(leining)
    return leinings

In [19]:
def namefixer(name):
    output = ""
    for char in name:
        if char.isalpha() == True:
            output += char
    return output

In [84]:
def make_command(item):
    name = item["name"]
    typ = item["type"]
    leiningname = "\\lein" + namefixer(name)
    command_lines = []
    aliyadict = {'1':"Kohen",'2':"Levi",'3':"Shelishi",'4':"Revi`i",'5':r"\d{H}amishi",'6':"Shishi",'7':"Shevi`i","M":"Maftir"}
    if typ=="weekday":
        leiningname+= "weekday"
    newcommand = r"\newcommand{"+leiningname+"}{"
    aliya_lines = []
    start_table = r"\begin{tabular}{c | c | c}"
    counter = 1
    for aliya in item["reading"].items():
        if "haft" not in aliya[0]:
            if aliya[0] == "3" and typ=="weekday":
                aliyaname = "Yisrael"
            else:
                aliyaname = aliyadict[aliya[0]]
            aliyacontent = make_verses(aliya[1])
            aliya_len = str(aliya[1]['v'])
            aliya_line = aliyaname + "&"+aliyacontent+"&"+aliya_len+r" pesukim\\\hline"
            aliya_lines.append(aliya_line)
    aliya_lines[-1] = aliya_lines[-1].replace(r"\hline","")
    end_table = r"\end{tabular}}"
    command_lines = [newcommand,start_table]
    for line in aliya_lines:
        command_lines.append(line)
    command_lines.append(end_table)
    if "haft" in item["reading"].keys():
        command_lines.append("\n")
        haft_name = "\\haft"+namefixer(item["reading"]["haftname"])
        haft_info = item["reading"]['haft']
        haft_command = r"\newcommand{"+haft_name+"}{"
        if type(item["reading"]['haftname']) == str:
            haft_text = "The Haftara for "+item["reading"]['haftname']+" is "
        elif type(item["reading"]['haftname']) == dict:
            haft_text = "The Haftara for "+item["reading"]['haftname']['haftara']+" is "
        else:
            print(item["reading"]['haftname'])
        if type(haft_info)==list:
            for haft_vs in haft_info:
                haft_text += make_verses(haft_vs) + r" \& "
            haft_text = haft_text[0:-3]
        else:
            verses = make_verses(haft_info)
            haft_text += verses
        haft_text += ". "
        haft_command += haft_text + "}\n"
        command_lines.append(haft_command)
    return command_lines
            

In [85]:
def make_verses(info):
    return info['k']+' '+info['b']+r'\textendash '+info['e']

In [86]:
info = get_data_from_jsons()
output = []
for item in info:
    output += make_command(item)


In [87]:
for line in output:
    print(line)

\newcommand{\leinRoshHashanaIonShabbat}{
\begin{tabular}{c | c | c}
Kohen&Genesis 21:1\textendash 21:4&4 pesukim\\\hline
Levi&Genesis 21:5\textendash 21:8&4 pesukim\\\hline
Shelishi&Genesis 21:9\textendash 21:12&4 pesukim\\\hline
Revi`i&Genesis 21:13\textendash 21:17&5 pesukim\\\hline
\d{H}amishi&Genesis 21:18\textendash 21:21&4 pesukim\\\hline
Shishi&Genesis 21:22\textendash 21:27&6 pesukim\\\hline
Shevi`i&Genesis 21:28\textendash 21:34&7 pesukim\\\hline
Maftir&Numbers 29:1\textendash 29:6&6 pesukim\\
\end{tabular}}


\newcommand{\haftRoshHashanaIonShabbat}{The Haftara for Rosh Hashana I (on Shabbat) is I Samuel 1:1\textendash 2:10. }

\newcommand{\leinRoshHashanaII}{
\begin{tabular}{c | c | c}
Kohen&Genesis 22:1\textendash 22:3&3 pesukim\\\hline
Levi&Genesis 22:4\textendash 22:8&5 pesukim\\\hline
Shelishi&Genesis 22:9\textendash 22:14&6 pesukim\\\hline
Revi`i&Genesis 22:15\textendash 22:19&5 pesukim\\\hline
\d{H}amishi&Genesis 22:20\textendash 22:24&5 pesukim\\\hline
Maftir&Numbers 2